In [101]:
# import basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [102]:
# load the datasets
steam = pd.read_csv('/kaggle/input/steam-store-games/steam.csv')
steam_description = pd.read_csv('/kaggle/input/steam-store-games/steam_description_data.csv')
steam_media = pd.read_csv('/kaggle/input/steam-store-games/steam_media_data.csv')
steam_requirement = pd.read_csv('/kaggle/input/steam-store-games/steam_requirements_data.csv')
steam_support = pd.read_csv('/kaggle/input/steam-store-games/steam_support_info.csv')
steamspy_tag = pd.read_csv('/kaggle/input/steam-store-games/steamspy_tag_data.csv')

# Initial Examination

In [103]:
steam.head(2)

In [104]:
steam_description.head(2)

In [105]:
steam_media.head(2)

In [106]:
steam_requirement.head(2)

In [107]:
steam_support.head(2)

In [108]:
steamspy_tag.head(2)

# EDA for each dataset

In [109]:
# check for missing values
steam.isna().sum()

No duplicates in this dataset.

In [110]:
steam.duplicated().sum()

In [111]:
steam.columns

In [112]:
steam.info()

Cleaning plan:

`categories`: split into lists

`genres`: split into lists

`owners`: get upper and lower bound, and then the median of them

`release_date`: convert to datetime format

### Split strings into lists

#### `categories`

In [113]:
# clean the values into lists
steam['categories'] = steam['categories'].apply(lambda x: x.split(";"))

# number of categories for each game
plt.figure()
steam['categories'].apply(lambda x: len(x)).value_counts().plot(kind = 'bar')
plt.xlabel('Number of Categories')
plt.ylabel('Frequency')
plt.title('Most Games only have a few Categories Listed')
plt.show()

In [114]:
# take a look at the categories
steam['categories']

In [115]:
# write a function to list all existing labels, and include all occurrences in another list
def get_labels(data):
  unique_labels = []
  all_labels = []

  for i in range(len(data)):
    for label in data[i]:
      all_labels.append(label)
      if label not in unique_labels:
        unique_labels.append(label)
  
  print('Number of unique labels:', len(unique_labels))
  print('Number of total labels applied:', len(all_labels))
  
  return unique_labels, all_labels

In [116]:
# list all existing categories, and include all occurrences in another list
unique_cats, all_cats = get_labels(steam['categories'])
unique_cats[:10]

Surprisingly, there's only 29 categories applied on 27,000+ games.. 

In [117]:
# visualize the top 10 categories
plt.figure(figsize = (14, 10))
pd.Series(all_cats).value_counts()[:10].plot(kind = 'barh')
plt.gca().invert_yaxis()
plt.xlabel('Frequency')
plt.title('Visualize the Frequency of Categories')
plt.show()

The most popular category is `Single-player`. There's more categories about players: `Multi-player`, `Local Multi-Player`, etc. 

Can we classify the games by the single/multiple player mode?

In [118]:
# encode categories
cats = steam['categories']
cats = cats.apply(lambda x: str(x).strip('[]'))
cats = cats.str.get_dummies(sep=', ')

In [119]:
# clean the column names
col_rename_dict = {}
for col_name in cats.columns:
  col_rename_dict[col_name] = col_name.strip("'")
cats.rename(columns = col_rename_dict, inplace = True)

In [120]:
# filter the categories about player modes
player_cols = [col for col in cats.columns if "layer" in col] # exclude p due to case sensitivity
player_cols

There's one category for single-player games and 4 for multi-player games.

However, the sub-categories of mult-player don't add up to `Multi-player`. 

In [121]:
# multi-player categories 
multi = cats[player_cols[:-1]]

# when Multi-player is 0, other sub-categories could still be 1
multi[multi['Multi-player'] == 0].sum(axis = 0)

In [122]:
# when Multi-player is 1, more than one sub-categories could be 1
multi[multi['Multi-player'] == 1].sum(axis = 1).value_counts()

Therefore, we'll just combine the sub-categories for multi-player games.

In [123]:
# when one of the multi-player subcategories is 1, the Multi-player column is 1
player_cats = cats[player_cols].copy()
player_cats.loc[(player_cats['Cross-Platform Multiplayer'] == 1) | (player_cats['Local Multi-Player'] == 1) | (player_cats['Online Multi-Player'] == 1), 'Multi-player'] = 1

In [124]:
player_cats = player_cats[['Multi-player', 'Single-player']]
player_cats.sum()

In [125]:
# single player only
s = len(player_cats[(player_cats['Single-player'] == 1) & (player_cats['Multi-player'] == 0)])
s

In [126]:
# single & multi player
s_m = len(player_cats[(player_cats['Single-player'] == 1) & (player_cats['Multi-player'] == 1)])
s_m

In [127]:
# multi player only
m = len(player_cats[(player_cats['Single-player'] == 0) & (player_cats['Multi-player'] == 1)])
m

In [128]:
# no player mode indicated
len(player_cats[(player_cats['Single-player'] == 0) & (player_cats['Multi-player'] == 0)])

Very few games have no player mode indicated. They are currently not included in the visualization.

In [129]:
# venn diagram
from matplotlib_venn import venn2 
venn2(subsets = (s, s_m, m), set_labels = ('Single-player', 'Multi-player'))
plt.title('Most Games Support Single-player Mode')
plt.show()

#### `genres`

In [130]:
# clean the values into lists
steam['genres'] = steam['genres'].apply(lambda x: x.split(";"))

In [131]:
# number of categories for each game
plt.figure()
steam['genres'].apply(lambda x: len(x)).value_counts().sort_index().plot(kind = 'bar')
plt.xlabel('Number of Genres')
plt.ylabel('Frequency')
plt.title('Most Games Have Less than 5 Genres Listed')
plt.show()

In [132]:
# list all existing genres, and include all occurrences in another list
unique_genres, all_genres = get_labels(steam['genres'])
unique_genres[:10]

In [133]:
# visualize the genres
plt.figure(figsize = (16, 10))
pd.Series(all_genres).value_counts().head(10).plot(kind = 'barh')
plt.gca().invert_yaxis()
plt.xlabel('Frequency')
plt.title('Visualize the Frequency of Genres')
plt.show()

#### `owners`

In [134]:
# split the lower and upper bound into a temperatory df
tmp = pd.DataFrame()
tmp[['lower', 'upper']] = steam['owners'].str.split('-', expand = True)
tmp['lower'] = tmp['lower'].astype('int')
tmp['upper'] = tmp['upper'].astype('int')
tmp['owner_median'] = (tmp['lower'] + tmp['upper']) / 2
tmp

In [135]:
# insert the owner_median column into steam df
steam['owner_median'] = tmp['owner_median']
steam.head(2)

In [136]:
# distribution of owner_median
plt.figure()
steam['owner_median'].value_counts().sort_index().plot(kind = 'barh')
plt.gca().invert_yaxis()
plt.xlabel('Frequency')
plt.ylabel('owner_median')
plt.title('Most Games are Owned by < 20k Players')
plt.show()

In [137]:
# top games by owners
steam.sort_values(by = 'owner_median', ascending = False).head(6)[['name', 'owner_median']]

### `release_date`

In [138]:
# convert release_date to datetime format
steam['release_date'] = steam['release_date'].astype('datetime64')

#### By year

Data gathered in May 2019, so 2019 has incomplete data (unexpectedly low releases).

In [139]:
# release date by year
plt.figure()
steam['release_date'].dt.year.value_counts().sort_index().plot(kind = 'line')
plt.ylabel('Releases')
plt.title('Releases Grow by Years')
plt.show()

In [140]:
# release date by year, exclude 2019
plt.figure()
steam['release_date'].dt.year.value_counts().sort_index()[:-1].plot(kind = 'line')
plt.ylabel('Releases')
plt.title('Releases Grow by Years')
plt.show()

#### By month

Most games are released around October, while very few games are released in June. 

In [141]:
# release date by month, exclude 2019
plt.figure()
steam[steam['release_date'] < '2019-01-01']['release_date'].dt.month.value_counts().sort_index().plot(kind = 'line')
plt.xlabel('Month')
plt.ylabel('Releases')
plt.title('Most Games are Released around October')
plt.show()

#### By trend over time

In [142]:
# plot by daily releases over time
release_counts = steam['release_date'].value_counts().sort_index()

plt.figure()
release_counts.plot()
plt.ylabel('Daily Releases')
plt.title('Not Many Releases Before 2006')
plt.show()

Not many releases before 2006.

In [143]:
# plot from 2006
plt.figure()
release_counts['2006-01-01':].plot()
plt.ylabel('Daily Releases')
plt.title('Too Messy to Specify Trends')
plt.show()

Too messy! Let's count by year-week.

In [144]:
from datetime import datetime
# reformat the release date to year-week
release_year_week = steam['release_date']
release_year_week = release_year_week.apply(lambda x: x.strftime("%Y-%W"))
release_year_week = release_year_week[release_year_week >= '2006-01'] # start from 2006
release_year_week

In [145]:
# plot releases by year-week
plt.figure(figsize = (10, 8))
release_year_week.value_counts().sort_index().plot()
plt.ylabel('Releases')
plt.title('Still A Little Messy')
plt.show()

Still a little messy. Let's try year-month.

In [146]:
# reformat the release date
release_year_month = steam['release_date']
release_year_month = release_year_month.apply(lambda x: x.strftime("%Y-%m"))
release_year_month = release_year_month[release_year_month >= '2006-01'] # start from 2006
release_year_month

In [147]:
# plot releases by year-month
plt.figure(figsize = (12, 8))
release_year_month.value_counts().sort_index()[:-1].plot() # Exclude the last month since the data was collected before the end of that month.
plt.ylabel('Releases')
plt.title('Good Overall Growth')
plt.show()

This looks good!!

#### highest daily release amount

On 2018-07-13, 64 games were releasedy!

In [148]:
# highest daily release amount
steam['release_date'].value_counts().sort_values(ascending = False).head()

### Categorical Columns

Next, we'll look at all the categorical columns together.

In [149]:
# categorical columns
cat_cols = ['english', 'developer', 'publisher', 'required_age']

# required_age is numeric but here it better suits as a categorical column

In [150]:
# plot all categorical columns
# note: only the top 10 frequent categories are plotted
for col in cat_cols:
  plt.figure(figsize = (10, 8))
  tmp = steam[col].value_counts()
  tmp.head(10).plot(kind = 'barh')
  plt.gca().invert_yaxis()
  plt.xlabel(col)
  plt.ylabel('Frequency')
  plt.title(col)
  plt.show()
  print(f'Total number of categories in {col}: {len(tmp)}')
  print(tmp.head(10))
  print('\n')

The classes in `english` and `required_age` are very unbalanced. Almost all games are in English, and almost all games have no age limits. 

Can show the top publishers and developers in final visualizations.

### Numeric Columns

In [151]:
# numeric columns
num_cols = ['achievements', 'positive_ratings', 'negative_ratings', 'average_playtime', 'median_playtime', 'price']

In [152]:
# visualize all numeric columns
for col in num_cols:
  plt.figure(figsize = (10, 8))
  steam[col].plot(kind = 'hist', bins = 50)
  plt.xlabel(col)
  plt.ylabel('Frequency')
  plt.title(col)
  plt.show()
  print('\n')

Um. The data is extremely skewed. Let's exclude the outliers and zoom in..

In [153]:
# write a function to remove outliers
# In the definition of box plots, values < Q1 - 1.5 IQR and values > Q3 + 1.5 IQR are the outliers
def remove_outliers(data):
  Q1 = data.quantile(0.25)
  Q3 = data.quantile(0.75)
  IQR = Q3-Q1
  data_outlier_excluded = data[(data >= Q1 - (1.5 * IQR)) & (data <= Q3 + (1.5 * IQR))]
  return data_outlier_excluded

#### `achievements`

In [154]:
# games with 0 achievements
len(steam[steam['achievements'] == 0])

In [155]:
# at least 1 achievements
steam[steam['achievements'] != 0]['achievements'].sort_values(ascending = False)

11864 games don't include any achivement.

There is a game with 9821 achievements! What is it?

In [156]:
# check the game with the most achievements
steam.loc[12604]

It's a puzzle game! You get achievements by finishing parts of the puzzle..amazing

There are too many outliers to get a meaningful distribution. Let's remove the outliers.

In [157]:
# exclude games with 0 achievements
none_0_ach = steam[steam['achievements'] != 0]['achievements']

# exclude the outliers
ach_ex = remove_outliers(none_0_ach)

# plot the distribution of achievements
plt.figure()
ach_ex.plot(kind = 'hist', bins = 50)
plt.xlabel('Number of Achievements')
plt.title('Most Games have <70 Achievements')
plt.show()

# conclusion
print('total games:', len(steam['achievements']))
print('games with non-zero achievements:', len(none_0_ach))
print('games with non-zero achievements, outliers excluded:', len(ach_ex))

About half of the games have no achievement system.

Most games have <70 achievements, while the champion has 9000+ achievements.

There's a small peak around 12 on the histogram.

#### `positive_ratings`

Let's look at the top 10 games with the most positive ratings.

In [158]:
# top 10 game by positive ratings
steam.sort_values(by = 'positive_ratings', ascending = False).head(10)[['name', 'positive_ratings']]

Now the distribution:

In [159]:
# plot the distribution of positive ratings
plt.figure()
steam['positive_ratings'].value_counts().sort_index().plot()
plt.xlabel('Positive Ratings')
plt.ylabel('Frequency')
plt.title('Need to Exclude the Outliers')
plt.show()

Um. Need to exclude outliers to make it human-interpretable.

In [160]:
# exclude outliers from positive_ratings
pos_r = steam['positive_ratings']
pos_r_ex = remove_outliers(pos_r)

# plot the distribution of positive ratings
plt.figure()
pos_r_ex.plot(kind = 'hist', bins = 50)
plt.xlabel('Positive Ratings')
plt.ylabel('Frequency')
plt.title('Most Games are not Frequently Rated')
plt.show()

# conclusion
print('total games:', len(pos_r))
print('games with positive ratings outliers excluded:', len(pos_r_ex))

This looks better.

Most Games are not frequently rated though.

#### `negative_ratings`

Top 10 with the most negative ratings

In [161]:
steam.sort_values(by = 'negative_ratings', ascending = False).head(10)[['name', 'negative_ratings']]

A LOT of overlap. Mostly because of popularity.

We should look at the ratio instead.

In [162]:
# ratio of positive/negative ratings
pos_neg_ratio = steam['positive_ratings'] / (steam['negative_ratings'] + 1) # +1 to avoid zero divisions
top_ratios_id = pos_neg_ratio.sort_values(ascending = False).head(5).index

steam.loc[top_ratios_id]

Now these games have high ratios but are not very popular..

Should still consider the total number of ratings at ranking!

Final solution: rank with number of positive ratings, but show negative ratings for these games at the same time. 

See the Tableau file for visualization.

#### `price`

In [163]:
# free games
steam[steam['price'] == 0].shape[0]

In [164]:
# pricy games
steam.sort_values(by = 'price', ascending = False).head()

Well, people don't seem to like to pay for such pricy games!

In [165]:
# exclude the outliers in price
price = steam['price']
price_ex = remove_outliers(price)

# plot the distribution of price
plt.figure()
price_ex.plot(kind = 'hist', bins = 20)
plt.xlabel('Price')
plt.title('Most Games Cost Less than £16')
plt.show()

# conclusion
print('total games:', len(steam))
print('games with price outliers excluded:', len(price_ex))
print('\n')
print('Note: the price data was collected in £.')

Most games are cheap.. 

But also, most games are not played much. 

What's the distribution of prices for games with non-zero playtime?

In [166]:
# plot the distribution of prices for games with non-zero playtime
plt.figure()
remove_outliers(steam[steam['average_playtime'] != 0]['price']).plot(kind = 'hist', bins = 20)
plt.xlabel('Price')
plt.title('Broader Price Range for Games with Longer Playtime')
plt.show()

After excluding the games with unnoticable playtime, the upper bound of price distribution is higher. 

As long as the game is interesting, players will be willing to spend more time as well as money on it!

### Further EDA

#### Revenue by release date

In [167]:
# calculate revenue by release date, owner_median, and price for each game
sales = pd.DataFrame()
sales['revenue'] = steam['owner_median'] * steam['price'] 

# label the release year-month of each game
sales['release_month'] = steam['release_date'].apply(lambda x: x.strftime("%Y-%m"))
sales

In [168]:
# plot sales amount by release week
plt.figure(figsize = (12, 8))
sales.groupby('release_month').sum()['revenue'].plot()
plt.ylabel('Revenue')
plt.title('Revenue by Release Time')
plt.show()

What happened at the end of 2017??

In [169]:
# attach the name column to find the games with highest revenue
sales.index = steam['name']
sales.sort_values(['revenue'], ascending = False).head()

It's PUBG!

There isn't a very obvious trend of "games become more and more porfitable" over the years.. but maybe the yearly sales are going up. We just don't have the data for that. We can only estimate based on the release date, but people buy good games after they are released for years. 

#### Sales amount by game

Note: some popular games are cheap/free, but there's payable DLC or upgrade available and we don't have data on how many owners purchased these contents. e.g. CSGO

In [170]:
# visualize the games with highest revenue
plt.figure()
sales.sort_values(by = 'revenue', ascending = False).head(10).plot(kind = 'barh')
plt.gca().invert_yaxis()
plt.xlabel('Revenue')
plt.title('PUBG Has the Highest Revenue')
plt.show()

In [171]:
# exclude PUBG to zoom in on the rest
plt.figure()
sales.sort_values(by = 'revenue', ascending = False).head(11)[1:].plot(kind = 'barh')
plt.gca().invert_yaxis()
plt.xlabel('Revenue')
plt.title('Games with Highest Revenue, Excluding PUBG')
plt.show()

## `steam_description`

There are 3 description columns in this table.

`detailed_description` and `about_the_game` are mostly identical.

In [172]:
print('Number of games with detailed_description and about_the_game the same:')
len(steam_description[steam_description['detailed_description'] == steam_description['about_the_game']])

The detailed description is on average 9.6x longer than the short one.

In [173]:
# compare the length of detailed/short description
print('The detailed description is this many times longer than the short description:')
round((steam_description['detailed_description'].str.len() / steam_description['short_description'].str.len()).mean(), 1)

### NLP 
Let's do NLP (natural language processing) on `short_description` to get the most popular words in the descriptions.

In [174]:
# clean the descriptions

# strip html
import re
steam_description['short_description'] = steam_description['short_description'].apply(lambda x: re.sub('<[^<]+?>', '', x))
steam_description['short_description'] = steam_description['short_description'].apply(lambda x: x.replace('&quot', '').replace('\r\n', ''))

# convert to lowercase
steam_description['short_description'] = steam_description['short_description'].apply(lambda x: x.lower())

In [175]:
# vectorize the words (top 100 with highest frequency)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words = 'english', min_df = 5, max_features = 100)
cv.fit(steam_description['short_description'])
short_des_transformed = cv.transform(steam_description['short_description'])

In [176]:
# extract the information and put it in a dataframe
short_des_vectorized = pd.DataFrame(columns = cv.get_feature_names(), 
                                    data = short_des_transformed.toarray())
short_des_vectorized.head(2)

In [177]:
# count the occurrence of each word (token)
tokens = short_des_vectorized.sum(axis = 0).sort_values(ascending = False)
tokens

In [178]:
# join the tokens into one string
tokens_str = ' '.join(tokens.index)
tokens_str

In [179]:
# create a word cloud
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(tokens_str)
 
# plot the word cloud image                      
plt.figure(figsize = (12, 12), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
 
plt.show()

Some words should be removed from the list to improve the image quality e.g. `game`, `player`..

Let's try for the top 100 popular games.

In [180]:
# id of the top 100 games by positive ratings
top_100_id = steam.sort_values(by = 'positive_ratings', ascending = False).head(100)['appid']
top_100_id.values

In [181]:
# short description of the top 100 games
top_100_des = pd.merge(top_100_id, steam_description, left_on = 'appid', right_on = 'steam_appid', how = 'left')['short_description']
top_100_des

In [182]:
# vectorize the top 100 words with highest frequency
cv = CountVectorizer(stop_words = 'english', min_df = 5, max_features = 100)
cv.fit(top_100_des)
top_100_des_transformed = cv.transform(top_100_des)

# extract the information and put it in a data frame to make it easier to see what has occured
top_100_des_vectorized = pd.DataFrame(columns = cv.get_feature_names(), 
                                    data = top_100_des_transformed.toarray())
top_100_des_vectorized.head(2)

In [183]:
# count the occurrence of each word (token)
tokens = top_100_des_vectorized.sum(axis = 0).sort_values(ascending = False)
tokens

# Conclusion

Table `steam` has the most information valuable for visualization. The Tableau visualization will include: 
* `categories`: most popular categories
* `genres`: most popular genres
* `owners`: most owned game, distribution of game owners
* `release_date`: total games released, releases by year, highest daily releases
* `developer`: developers released most games
* `publisher`: publishers released most games
* `positive_ratings` & `negative_ratings`: average pos:neg ratio, top games by positive ratings with negative ratings shown at the side
* `average_playtime` & `median_playtime`: most played game, distribution of playtime
* total revenue, revenue by release date

Table `steam_description` can be studied with NLP to extract the most frequent words used in popular games, in order to show possible trends.
* Columns `detailed_description` and `about_the_game` are mostly identical
* `short_description` is more concise and will be used for extraction.

Table `steamspy_tag` will be used to visualize the most popular tags voted by community.


The information in tables `steam_description` and `steamspy_tag` will be first processed by a python file. The output files will then be visualized in Tableau.

See my Git Hub for the full work:

https://github.com/aaw707/Steam-Games-Visualization